In [1]:
import mesa
from mesa import Model
from mesa.datacollection import DataCollector
from mesa.space import MultiGrid
from mesa.time import RandomActivation
from random import randrange, choices, sample
import random

# Functions

In [2]:
def generate_probability():
    return randrange(100)

def generate_weights():
    pass

def generate_score():
    return randrange(100)

def generate_distribution(n, total):
    dividers = sorted(sample(range(1, total), n - 1))
    return [a - b for a, b in zip(dividers + [total], [0] + dividers)]



class Behavior:
    ActionSuccessProbability = generate_probability()
    TakeActionProbability = generate_probability()
    ActionSelectionWeights = generate_weights()
    MakeMoveProbability = generate_probability()
    BreedSuccessProbability = generate_probability()
    BreedActionProbability = generate_probability()
    MoveDirectionDistribution = generate_distribution(9, 100)
    AwarenessScore = generate_score()
    ActionOrderDistribution = generate_distribution(3, 100)
    ActionPrivilegeScore = generate_score()
    HappinessScore = generate_score()
    EnvironmentScore = generate_score()
    
    def fate(self, weight):
        fate_list = [True, False]
        weight_var = (weight, 100-weight)

        return random.choices(fate_list, weights=weight_var, k=len(fate_list))[0]

In [3]:
class RandomWalker(mesa.Agent):
    """
    Class implementing random walker methods in a generalized manner.
    Not intended to be used on its own, but to inherit its methods to multiple
    other agents.
    """

    grid = None
    x = None
    y = None
    moore = True

    def __init__(self, unique_id, pos, model, moore=True):
        """
        grid: The MultiGrid object in which the agent lives.
        x: The agent's current x coordinate
        y: The agent's current y coordinate
        moore: If True, may move in all 8 directions.
                Otherwise, only up, down, left, right.
        """
        super().__init__(unique_id, model)
        self.pos = pos
        self.moore = moore

    def random_move(self):
        """
        Step one cell in any allowable direction.
        """
        # Pick the next cell from the adjacent cells.
        next_moves = self.model.grid.get_neighborhood(
            self.pos, self.moore, True)

        # move check
        if len(next_moves) > 0:

            if self.behavior.fate(self.behavior.MakeMoveProbability):
                next_move = self.random.choices(
                    next_moves, weights=self.behavior.MoveDirectionDistribution, k=1)[0]
                self.model.grid.move_agent(self, next_move)

            else:
                pass

        else:
            self.model.grid.remove_agent(self)
            self.model.schedule.remove(self)

In [4]:
class Dooder(RandomWalker):
    """
    """
    def __init__(self, unique_id, pos, model):
        super().__init__(unique_id, pos, model)
        self.behavior = Behavior()
    
    def kill(self, agent):
        self.model.grid.remove_agent(agent)
        self.model.schedule.remove(agent)

    def die(self):
        self.model.grid.remove_agent(self)
        self.model.schedule.remove(self)

    def step(self):
        # move --> action
        print(self.unique_id)
        self.random_move()

In [35]:
class Simulation(Model):
    """
    Wolf-Sheep Predation Model
    """

    height = 20
    width = 20

    initial_agents = 10
    verbose = True  # Print-monitoring

    description = (
        "A model for simulating wolf and sheep (predator-prey) ecosystem modelling."
    )

    def __init__(
        self,
        width=20,
        height=20,
        initial_agents=initial_agents,
        verbose = verbose
    ):
        """
        """
        super().__init__()
        # Set parameters
        self.width = width
        self.height = height
        self.initial_agents = initial_agents
        self.verbose = verbose

        self.schedule = RandomActivation(self)
        self.grid = MultiGrid(self.width, self.height, torus=True)
        self.datacollector = DataCollector({"Dooders": lambda m: m.schedule.get_agent_count()})
        self.agent_count = 0

        # Create agents:
        for i in range(self.initial_agents):
            x = self.random.randrange(self.width)
            y = self.random.randrange(self.height)
            happiness = self.random.randrange(10,90)
            dooder = Dooder(self.next_id(), (x, y), self)
            self.grid.place_agent(dooder, (x, y))
            self.schedule.add(dooder)

        self.running = True
        self.datacollector.collect(self)

    def step(self):
        self.schedule.step()
        # collect data
        self.datacollector.collect(self)
        if self.verbose:
            print([self.schedule.time, self.schedule.get_agent_count()])

    def run_model(self, step_count=10):

        if self.verbose:
            print("Initial number dooders: ", self.schedule.get_agent_count())

        for i in range(step_count):
            self.agent_count = self.schedule.get_agent_count()
            # running status
            # metrics
            self.step()

        if self.verbose:
            print("")
            print("Final number dooders: ", self.schedule.get_agent_count())

In [36]:
test = Simulation()

In [37]:
test.current_id

10

In [38]:
test.run_model()

Initial number dooders:  10
6
2
9
4
8
1
5
7
10
3
[1, 10]

Final number dooders:  10


In [39]:
test.running

True

In [40]:
# How to stop a model in progress
# Does the class save actions taken?

In [41]:
test.schedule.agents

In [42]:
test.schedule.agents[1].pos

(9, 4)

In [43]:
test.grid.get_neighbors((10,16), moore=True)

[<__main__.Dooder at 0x2363ec90a00>, <__main__.Dooder at 0x2363ec90ac0>]

In [44]:
test.datacollector.get_agent_vars_dataframe()

,
Step,AgentID


In [45]:
test.datacollector.get_model_vars_dataframe()

,Dooders
0,10
1,10


In [46]:
test.datacollector.tables

{}

In [47]:
test.grid.torus

True